In [1]:
from IPython import display
from IPython.core.display import HTML

from mpl_toolkits import mplot3d

import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

import imageio

In [2]:
np.random.seed(972)

## Задача

При помощи генетического алгоритма найти один из локальных минимумов функции Химмельблау

https://ru.wikipedia.org/wiki/%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F_%D0%A5%D0%B8%D0%BC%D0%BC%D0%B5%D0%BB%D1%8C%D0%B1%D0%BB%D0%B0%D1%83

$z = (x ^ 2 + y - 11) ^ 2 + (x + y ^ 2 - 7) ^ 2$

In [3]:
def func(x, y):
    return (x ** 2 + y - 11) ** 2 + (x + y ** 2 - 7) ** 2

График функции:

In [4]:
x = np.linspace(-5, 5, 30)
y = np.linspace(-5, 5, 30)

X, Y = np.meshgrid(x, y)
Z = func(X, Y)

In [5]:
fig = plt.figure(figsize=(16, 6))

images = []

for i in range(0, 360, 1):
    plt.clf()

    ax = fig.add_subplot(1, 2, 1, projection='3d')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.plot_surface(
        X, Y, Z, rstride=1, cstride=1, cmap='winter', edgecolor='none'
    )
    ax.set_title("Функция Химмельблау")

    ax.view_init(40, i)
    plt.draw()
    
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(image)
    
    display.clear_output(wait=True)
    display.display(plt.gcf())
    
imageio.mimsave('function_graph_ru.gif', images, fps=30)

plt.close()
display.clear_output(wait=True)
HTML('<img src="function_graph_ru.gif">')

Одна популяция состоит из 4 особей

In [6]:
POPULATION_SIZE = 4

In [7]:
population = np.random.uniform(-6, 6, [POPULATION_SIZE, 2])
population

array([[ 5.91979742,  4.86662271],
       [ 0.52676503, -2.35037198],
       [-0.79943434,  3.55565701],
       [-5.52252641,  3.50357636]])

### Функция приспособленности

In [8]:
def calc_fitness(population) -> float:
    return 1 / (np.sqrt(np.square(0 - func(*population[:].T))) + 1e-200)

### Отбор
Отбор особей будет производиться методом рулетки — вероятность выбора особи тем выше, чем больше значение функции  приспособленности $p_{i}={\frac {f_{i}}{\sum _{i=1}^{N}{f_{i}}}}$, где $p_{i}$ — вероятность выбора $i$ особи, $f_{i}$ — значение функции приспособленности для $i$ особи, $N$ — количество особей в популяции

In [9]:
def select_population(population) -> np.ndarray:
    fitness = calc_fitness(population)
    probabilities = fitness / fitness.sum()
    selected_indexes = np.random.choice(
        np.arange(POPULATION_SIZE), 3, p=probabilities, replace=False
    )
    selected_population = population[selected_indexes]
    selected_population_fitness = calc_fitness(selected_population)
    sorted_indexes = sorted(
        enumerate(selected_population_fitness), 
        key=lambda x: x[1], reverse=True
    )
    sorted_indexes = np.array(sorted_indexes).astype(np.uint8)
    return selected_population[sorted_indexes[:, 0]]

### Скрещивание

$(a_1, a_2) \rightarrow (b_1, a_2)$  
$(b_1, b_2) \rightarrow (c_1, a_2)$  
$(c_1, c_2) \rightarrow (a_1, b_2)$  
$(d_1, d_2) \rightarrow (a_1, c_2)$

In [10]:
def cross_population(selected_population) -> np.ndarray:
    result = np.zeros([POPULATION_SIZE, 2])
    
    result[:2, 1] = selected_population[0, 1]
    result[2:, 0] = selected_population[0, 0]
    
    result[0, 0] = selected_population[1, 0]
    result[2, 1] = selected_population[1, 1]
    
    result[1, 0] = selected_population[2, 0]
    result[3, 1] = selected_population[2, 1]
    
    return result

### Мутации

Для поддержания разнообразия популяции необходимо проведение мутации, состоящее в добавлении к некоторым элементам случайных значений

In [11]:
def apply_mutations(population) -> None:
    mutations = np.random.randint(0, 2, population.shape) * \
        np.random.uniform(-0.25, 0.25, population.shape)
    population += mutations

In [12]:
def train(population, generations_num, verbose=1, 
          extreme_expected_value=None) -> tuple:
    best_population = np.copy(population)
    best_fitness = float('-inf')
    best_solutions = []
    
    for generation_index in tqdm(range(generations_num)):
        fitness = calc_fitness(population)
        selected_population = select_population(population)
        best_solution = population[fitness.argmax()]
        
        if max(fitness) > best_fitness:
            best_fitness = max(fitness)
            best_population = np.copy(population)
            best_solutions.append(best_solution)
        
        func_value = func(*best_solution)
        if extreme_expected_value and func_value == extreme_expected_value:
            break

        population = cross_population(selected_population)
        apply_mutations(population)
        
        if (generation_index + 1) % verbose == 0:
            print(
                f'Поколение {generation_index + 1}, '
                f'лучшее значение функции приспособленности = {best_fitness:.4f}, '
                f'минимальное найденное значение функции = {min(func(*best_population.T)):.12f}'
            )
    return best_fitness, best_population, best_solutions

In [13]:
best_fitness, best_population, best_solutions = train(
    population, generations_num=10000000, verbose=1000000, 
    extreme_expected_value=0
)

Поколение 1000000, лучшее значение функции приспособленности = 1224991.0207, минимальное найденное значение функции = 0.000000816333
Поколение 2000000, лучшее значение функции приспособленности = 1334895.5681, минимальное найденное значение функции = 0.000000749122
Поколение 3000000, лучшее значение функции приспособленности = 2329613.4168, минимальное найденное значение функции = 0.000000429256
Поколение 4000000, лучшее значение функции приспособленности = 2831066.2471, минимальное найденное значение функции = 0.000000353224
Поколение 5000000, лучшее значение функции приспособленности = 2831066.2471, минимальное найденное значение функции = 0.000000353224
Поколение 6000000, лучшее значение функции приспособленности = 4341275.3798, минимальное найденное значение функции = 0.000000230347
Поколение 7000000, лучшее значение функции приспособленности = 4341275.3798, минимальное найденное значение функции = 0.000000230347
Поколение 8000000, лучшее значение функции приспособленности = 434127

Визуализация процесса обучения:

In [14]:
fig, ax = plt.subplots(1, 1)
plt.title('Процесс обучения')
plt.xlabel('x')
plt.ylabel('y')
cp = ax.contourf(X, Y, Z, 40)
fig.colorbar(cp)

images = []

for i in range(0, len(best_solutions) - 1, 4):
    x, y = best_solutions[i][0], best_solutions[i][1]
    dx, dy = best_solutions[i+1][0] - x, best_solutions[i+1][1] - y
    plt.scatter(*best_solutions[i], c='r', linewidths=0.1)
    plt.draw()
    
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    images.append(image)
    
    display.clear_output(wait=True)
    display.display(plt.gcf())

imageio.mimsave('learning_ru.gif', images, fps=3)

plt.close()
display.clear_output(wait=True)
HTML('<img src="learning_ru.gif">')

Функция Химмельблау имеет 4 равнозначных локальных минимума:

* $z(3, 2) = 0$  
* $z(-2.8..., 3.1...) = 0$  
* $z(-3.779..., -3.28...) = 0$
* $z(3.58..., -1.8...) = 0$

Минимальное значение, обнаруженное в результате вполнения алгоритма:

In [15]:
print(f'z{tuple(best_solutions[-1])} = {func(*best_solutions[-1]):.12f}')

z(3.000059549235091, 2.00000183564038) = 0.000000133452
